In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

TRAIN_PATH = "../input/dogs-vs-cats/train.zip"
TEST_PATH = "../input/dogs-vs-cats/test1.zip"

FILES = "/kaggle/files/unzipped/"

with zipfile.ZipFile(TRAIN_PATH, 'r') as zipp:
    zipp.extractall(FILES)
    
with zipfile.ZipFile(TEST_PATH, 'r') as zipp:
    zipp.extractall(FILES)
    
os.listdir('/kaggle/files/unzipped/')

In [ ]:
import tensorflow as tf


In [ ]:
import pandas as pd

training_images_files=os.listdir("/kaggle/files/unzipped/train")
test_image_files=os.listdir("/kaggle/files/unzipped/test1")

classes=[]
for f in training_images_files:
    class_name=f.split('.')[0]
    classes.append(class_name)

train_df = pd.DataFrame({
    'filename': training_images_files,
    'class': classes
})


train_df=train_df.reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, validate_df = train_test_split(train_df, test_size=0.20)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
            rescale=1/255.,
            rotation_range=45,
            shear_range=0.1,
            zoom_range=0.2,
            horizontal_flip=True,
            width_shift_range=0.1,
            height_shift_range=0.1
)

train_generator=train_datagen.flow_from_dataframe(
            train_df,
            '/kaggle/files/unzipped/train/',
            x_col='filename',
            y_col='class',
            target_size=(299,299),
            batch_size=128,
            class_mode='binary'

)

validation_datagen=ImageDataGenerator(
            rescale=1/255
)

validation_generator=validation_datagen.flow_from_dataframe(
            validate_df,
            '/kaggle/files/unzipped/train/',
            x_col='filename',
            y_col='class',
            target_size=(299,299),
            batch_size=128,
            class_mode='binary'

)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (299, 299, 3),
                                include_top = False,
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
    layer.trainable=False

pre_trained_model.summary()

In [ ]:
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

last_layer = pre_trained_model.get_layer('mixed10')

last_output=last_layer.output

def create_model(last_output):
    x=keras.layers.Flatten()(last_output)
    x=keras.layers.Dense(128, activation='relu')(x)
    x=keras.layers.Dropout(0.4)(x)
    x=keras.layers.Dense(64, activation='relu')(x)
    x=keras.layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(optimizer = RMSprop(learning_rate=0.001),
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])
    return model



model=create_model(last_output)
    

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_delt=0.001,
                                            min_lr=0.00001)

early_stopping = EarlyStopping(
    monitor = "val_accuracy",
    patience = 50,
    verbose = 1,
    mode = "max",
)

In [ ]:


tf.keras.backend.clear_session()

history=model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[learning_rate_reduction, early_stopping]
)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random

test_images=[]
test_image_files=random.sample(test_image_files, 500)
for test_image in test_image_files:
    test_images.append(img_to_array(load_img('/kaggle/files/unzipped/test1/'+test_image, target_size=(299,299)))/255.)

In [ ]:
test_images=np.array(test_images)

In [ ]:
predictions=model.predict(test_images)

In [ ]:
from matplotlib import pyplot as plt

i=random.randint(0, len(predictions))
print(predictions[i])
if predictions[i] >0.5:
    print('Its a Dog')
else:
    print('Its a Cat')
          

plt.imshow(test_images[i], interpolation='nearest')
plt.show()